In [26]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
import math
import scipy.integrate as si
import scipy.optimize as so

In [27]:
# PARAMETERS

concentration = 10 # concentration of pre-encounter minihalo
Efrac_min = 1e-4 # minimum value of E_frac to be considered
Efrac_max = 1e3  # maximum value of E_frac to be considered
num_Efracs = 41 # total number of E_frac values to be considered, logarithmically spaced between 'Efrac_min' and 'Efrac_max'
save_to_disk = False

In [28]:
def second_derivative_parametric_input(x, y, t):
    # Input should be 'x' in terms 't', and 'y' in terms of 't'
    # Output is second derivative of 'y' with respect to 'x' ('d2y/dx2') as a function of 't'
    dy_dx = sp.Derivative(y, t) / sp.Derivative(x, t)
    d2y_dx2 =  sp.Derivative(dy_dx, t) / sp.Derivative(x, t)
    d2y_dx2_in_terms_of_t = sp.nsimplify(d2y_dx2.doit())


    return d2y_dx2_in_terms_of_t
    

In [29]:
c, f_NFW, x_prime = sp.symbols("c f_NFW x_prime", positive=True)

# For NFW AMC

crho_NFW = 1 / (c*x_prime) / (1+c*x_prime)**2 # dimensionless density of NFW profile. 'chro_NFW = rho_NFW / rho_s', where 'rho_NFW' is the density profile and 'rho_s' is the scale radius.
psi_2_NFW = 1/f_NFW * ( sp.log(1+c*x_prime)/x_prime - c/(1+c*x_prime) )  # normalized relative potential. From equation (25) in our article

d2crho_dpsi2_NFW = second_derivative_parametric_input(psi_2_NFW, crho_NFW, x_prime) # Second derivative of 'chro_NFW' with respect to 'psi_2_NFW' as a functin of 'x_prime'


# For Hernquist AMC

crho_Hern = 1 / (c*x_prime) / (1+c*x_prime)**3 # dimensionless density of Hernquist profile. 'chro_Hern = rho_Hern / rho_1', where 'rho_Hern' is the density profile and 'rho_1' is the scale radius.
psi_2_Hern = (1+c)**2 * x_prime / (1+c*x_prime)**2 # normalized relative potential. From equation (30) in our article

d2crho_dpsi2_Hern = second_derivative_parametric_input(psi_2_Hern, crho_Hern, x_prime)  # Second derivative of 'chro_Hern' with respect to 'psi_2_Hern' as a functin of 'x_prime'

In [30]:
c_val = concentration      #Initializing the concentration of the unperturbed NFW AMC

# NFW functions

# compute normalized crossover radius 'x*'
def get_x_star_NFW():
    eqn = lambda x_c: beta / alpha_squared * E_frac * x_c**2 - ( 1/f_NFW_val * ( np.log(1+c_val*x_c) / x_c  -  c_val / (1+c_val*x_c) ) )
    sol = so.fsolve(eqn, 1)[0]
    
    return sol

# Computes normalized radius 'x' given a value of normalized relative potential 'psi_B', using equation (25) in our article
def get_x_prime_NFW(psi_prime):
    eqn = lambda x_Prime: psi_prime  -  1/f_NFW_val * ( np.log(1+c_val*x_Prime)/x_Prime - c_val/(1+c_val*x_Prime) )
    sol = so.root_scalar(eqn, bracket=(x_star, 1e20)).root
    
    return sol

# Compute the integrand according to equation (J8) in our article
def integrand_1_NFW(psi_prime, eps, x):
    
    psi_1_NFW = 1/f_NFW_val * ( math.log(1+c_val*x)/x - c_val/(1+c_val*x_star) )
    x_prime = get_x_prime_NFW(psi_prime)

    return (
        x**2 * 1/(math.sqrt(8)*math.pi**2) * 1/np.sqrt(eps - psi_prime) * 
        d2crho_dpsi2_c_and_fNFW_substituted_lambdified_NFW(x_prime).real * np.sqrt(2*(psi_1_NFW - eps))
    )



def range_x_1_NFW_I():
    return [0, x_star]


# Hernquist functions

# compute the concentratin of the Hernquist profile given its scale density
def compute_c_Hern(rho):
    eqn = lambda c_Hern: 1 / 2 / c_Hern / (1 + c_Hern)**2   -   200 / 3 * rho_crit / rho
    sol = so.fsolve(eqn, 1)[0]
    return sol



# Functions common for both NFW and Hernquist AMCs. Therefore, no suffix.

def range_eps_1(x):
    return [0, beta / alpha_squared * E_frac * x**2]

def range_psi_prime_1(eps, x):
    return [0, eps]



#------------------------------------------------------------------------




rho_crit = 9.1275e-27 # in kg/m^3 . This is the cosmological critical density

E_fracs = np.geomspace(Efrac_min, Efrac_max, num_Efracs)  # Creates a logarithmically spaced array of E_frac values. Adjust as necessary


f_NFW_val = math.log(1+c_val) - c_val/(1+c_val) # using equation (14) in our article 

# Scale densities of AMCs

rho_s = 200/3 * c_val**3/(np.log(1+c_val) - c_val/(1+c_val)) * rho_crit # kg/m^3 . Scale density of of NFW minihalo

survival_fractions = np.zeros(np.size(E_fracs)) # Create a zeros array the same size as the 'E_fracs' array. This will eventually contain the values of survival fractions corresponding to E_frac values.
x_stars = np.zeros(np.size(E_fracs)) 

for i in range(np.size(E_fracs)):

    E_frac = E_fracs[i]

    
    alpha_squared = 3/c_val**2 + 1/(2*f_NFW_val)*(c_val-3)/(c_val+1)
    beta = (c_val**3 - 2*c_val*(1+c_val)*f_NFW_val) / (2*(1+c_val)**2 * f_NFW_val**2)  # this is actually gamma in our article

    d2crho_dpsi2_c_and_fNFW_substituted_NFW = d2crho_dpsi2_NFW.subs({c:c_val, f_NFW:f_NFW_val})  # substitute the numerical values of 'concentration' and 'f_NFW' in 'd2crho_dpsi2_NFW'
    d2crho_dpsi2_c_and_fNFW_substituted_lambdified_NFW = sp.lambdify(x_prime, d2crho_dpsi2_c_and_fNFW_substituted_NFW)  # Lambdify 'd2crho_dpsi2_c_and_fNFW_substituted_NFW' so that you can evaluate it by passing a numerical
                                                                                                                        # value of 'x_prime' as input

    # Compute the normalized crossover radius 'x*'
    x_star = get_x_star_NFW()
    x_stars[i] = x_star

    # Compute concentration of first Hernquist AMC
    K = np.log(1 + c_val*x_star) - c_val*x_star/(1 + c_val*x_star) # this is 'f_NFW(c_s * x_s*)'
    I = si.nquad(integrand_1_NFW, [range_psi_prime_1, range_eps_1, range_x_1_NFW_I])[0] # from equation (J8) in our article
    R_s = np.sqrt(2*K - 8*np.pi*c_val**3*I) #from equation (45) in our article
    rho_1 = rho_s / R_s # from equation (42) in our article
    c_1 = compute_c_Hern(rho_1) # compute the concentratin of the Hernquist profile given its scale density

    x_1_rVirS = c_val/c_1 / R_s # equation (K3) in our article
    H = (c_1*x_1_rVirS)**2 / (1 + c_1*x_1_rVirS)**2 # this is just 'f_Hern(c_1*x_1_rVirS)'
    survival_fraction = 1/2 * R_s**2 * H/f_NFW_val # equation (K9) in our article
    survival_fractions[i] = survival_fraction

    print(f"Iteration #{i+1}")



Iteration #1
Iteration #2
Iteration #3
Iteration #4
Iteration #5
Iteration #6
Iteration #7
Iteration #8
Iteration #9
Iteration #10
Iteration #11
Iteration #12
Iteration #13
Iteration #14
Iteration #15
Iteration #16
Iteration #17
Iteration #18
Iteration #19
Iteration #20
Iteration #21
Iteration #22
Iteration #23
Iteration #24
Iteration #25
Iteration #26
Iteration #27
Iteration #28
Iteration #29
Iteration #30
Iteration #31
Iteration #32
Iteration #33
Iteration #34
Iteration #35
Iteration #36
Iteration #37
Iteration #38
Iteration #39
Iteration #40
Iteration #41


In [31]:
# Save Efrac array and survival fractions array to disk.
if save_to_disk:
    np.save("Efracs_after_relaxation", E_fracs)
    np.save(f"survival_fractions_after_relaxation_c_{c_val}", survival_fractions)